In [1]:
import numpy as np
import os
import cv2
import tqdm
import six.moves.urllib as urllib
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
import tensorflow as tf

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
%matplotlib inline
from PIL import Image

sys.path.append('models/research')
sys.path.append('models/research/object_detection')
from object_detection.utils import ops as utils_ops
from utils import label_map_util
from utils import visualization_utils as vis_util

In [2]:
ls data_dir/mixed_experiment/checkpoints_v2/

checkpoint                             model.ckpt-44909.data-00000-of-00001
events.out.tfevents.1567337207.miptai  model.ckpt-44909.index
events.out.tfevents.1567337618.miptai  model.ckpt-44909.meta
events.out.tfevents.1567401829.miptai  model.ckpt-45524.data-00000-of-00001
graph.pbtxt                            model.ckpt-45524.index
model.ckpt-43678.data-00000-of-00001   model.ckpt-45524.meta
model.ckpt-43678.index                 model.ckpt-46142.data-00000-of-00001
model.ckpt-43678.meta                  model.ckpt-46142.index
model.ckpt-44292.data-00000-of-00001   model.ckpt-46142.meta
model.ckpt-44292.index                 pipeline.config
model.ckpt-44292.meta


In [3]:
!nvidia-smi

Mon Sep  2 12:48:24 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.40       Driver Version: 430.40       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:19:00.0 Off |                  N/A |
|  0%   33C    P8     9W / 250W |  10775MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:1A:00.0 Off |                  N/A |
| 35%   64C    P0    81W / 250W |     18MiB / 11176MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [4]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "1"

In [9]:
# iference graph extraction
!export PYTHONPATH=$PYTHONPATH:models/research:models/research/slim;\
python models/research/object_detection/export_inference_graph.py \
--input_type=image_tensor \
--pipeline_config_path=data_dir/mixed_experiment/tf_api_v2.config \
--trained_checkpoint_prefix=data_dir/mixed_experiment/checkpoints_v2/model.ckpt-46142 \
--output_directory=data_dir/mixed_experiment/graphs_v2

/home/alexeygrigoriev/miniconda/envs/tensorflow_gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/alexeygrigoriev/miniconda/envs/tensorflow_gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/alexeygrigoriev/miniconda/envs/tensorflow_gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 

I0902 10:36:10.797902 139911306749760 regularizers.py:98] Scale of 0 disables regularizer.
I0902 10:36:10.971033 139911306749760 regularizers.py:98] Scale of 0 disables regularizer.
I0902 10:36:11.390850 139911306749760 regularizers.py:98] Scale of 0 disables regularizer.
W0902 10:36:11.468999 139911306749760 deprecation.py:323] From /home/alexeygrigoriev/Projects/TD/models/research/object_detection/exporter.py:362: get_or_create_global_step (from tensorflow.contrib.framework.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Please switch to tf.train.get_or_create_global_step
W0902 10:36:11.471194 139911306749760 deprecation.py:323] From /home/alexeygrigoriev/Projects/TD/models/research/object_detection/exporter.py:518: print_model_analysis (from tensorflow.contrib.tfprof.model_analyzer) is deprecated and will be removed after 2018-01-01.
Instructions for updating:
Use `tf.profiler.profile(graph, run_meta, op_log, cmd, options)`. Bu

240 ops no flops stats due to incomplete shapes.
Parsing Inputs...
Incomplete shape.

=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          .*BatchNorm.*,.*Initializer.*,.*Regularizer.*,.*BiasAdd.*
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================
Incomplete shape.

Doc:
scope: The nodes in the model graph are organiz

W0902 10:36:12.245284 139911306749760 deprecation_wrapper.py:119] From /home/alexeygrigoriev/Projects/TD/models/research/object_detection/exporter.py:411: The name tf.train.Saver is deprecated. Please use tf.compat.v1.train.Saver instead.

2019-09-02 10:36:12.887960: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcuda.so.1
2019-09-02 10:36:12.910618: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1640] Found device 0 with properties: 
name: GeForce GTX 1080 Ti major: 6 minor: 1 memoryClockRate(GHz): 1.6325
pciBusID: 0000:1a:00.0
2019-09-02 10:36:12.910807: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcudart.so.10.0
2019-09-02 10:36:12.912226: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcublas.so.10.0
2019-09-02 10:36:12.913316: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened d

I0902 10:36:16.426923 139911306749760 graph_util_impl.py:364] Converted 275 variables to const ops.
2019-09-02 10:36:16.637409: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1640] Found device 0 with properties: 
name: GeForce GTX 1080 Ti major: 6 minor: 1 memoryClockRate(GHz): 1.6325
pciBusID: 0000:1a:00.0
2019-09-02 10:36:16.637491: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcudart.so.10.0
2019-09-02 10:36:16.637505: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcublas.so.10.0
2019-09-02 10:36:16.637516: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcufft.so.10.0
2019-09-02 10:36:16.637526: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcurand.so.10.0
2019-09-02 10:36:16.637536: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfull

In [5]:
ls data_dir/mixed_experiment/graphs_v2

checkpoint                      model.ckpt.index  saved_model/
frozen_inference_graph.pb       model.ckpt.meta
model.ckpt.data-00000-of-00001  pipeline.config


In [6]:
PATH_TO_FROZEN_GRAPH = 'data_dir/mixed_experiment/graphs_v2/frozen_inference_graph.pb'

detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')
        
PATH_TO_LABELS = 'data_dir/mixed_experiment/annotations/label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

W0902 12:48:30.063549 139635363989312 deprecation_wrapper.py:119] From models/research/object_detection/utils/label_map_util.py:137: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



In [7]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)


def run_inference_for_single_image(image, graph):
    with graph.as_default():
        with tf.Session() as sess:
            # Get handles to input and output tensors
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in [
                    'num_detections', 'detection_boxes', 'detection_scores',
                    'detection_classes', 'detection_masks'
            ]:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                            tensor_name)
            if 'detection_masks' in tensor_dict:
                # The following processing is only for single image
                detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
                detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
                # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
                real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
                detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
                detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
                detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                        detection_masks, detection_boxes, image.shape[1], image.shape[2])
                detection_masks_reframed = tf.cast(
                        tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                # Follow the convention by adding back the batch dimension
                tensor_dict['detection_masks'] = tf.expand_dims(
                        detection_masks_reframed, 0)
            image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

            # Run inference
            output_dict = sess.run(tensor_dict, feed_dict={image_tensor: image})

            # all outputs are float32 numpy arrays, so convert types as appropriate
            output_dict['num_detections'] = int(output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict[
                    'detection_classes'][0].astype(np.int64)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]
            if 'detection_masks' in output_dict:
                output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict

In [8]:
PATH_TO_TEST_IMAGES_DIR = 'data_dir/test_images/random_stuff/mixed_images'
images = [x for x in os.listdir(PATH_TO_TEST_IMAGES_DIR) if x[-4:]=='.jpg']
TEST_IMAGE_PATHS = [os.path.join(PATH_TO_TEST_IMAGES_DIR, x) for x in images]
SAVE_DIR = 'data_dir/test_images/random_stuff/mixed_eval_images_v2/'

In [9]:
# draw GT for evaluated images
for image_path in tqdm.tqdm(TEST_IMAGE_PATHS):
    image = Image.open(image_path).convert('RGB')#.resize((596, 842)) 
    image_np = load_image_into_numpy_array(image)
    image_np_expanded = np.expand_dims(image_np, axis=0)
    output_dict = run_inference_for_single_image(image_np_expanded, detection_graph)
    raw_image = Image.open(image_path.replace('mixed_images', 'images')).convert('RGB').resize((596, 842)) 
    raw_image_np = load_image_into_numpy_array(raw_image)
    vis_util.visualize_boxes_and_labels_on_image_array(
        raw_image_np,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        use_normalized_coordinates=True,
        line_thickness=8,
        min_score_thresh=0.6)
    #plt.figure(figsize=IMAGE_SIZE)
    #plt.imshow(image_np)
    im = Image.fromarray(raw_image_np)
    im.save(SAVE_DIR+image_path.split('/')[-1])

100%|██████████| 34/34 [01:39<00:00,  2.93s/it]


In [13]:
PATH_TO_TEST_IMAGES_DIR = 'data_dir/test_images/TableBank/mixed_images'
images = [x for x in os.listdir(PATH_TO_TEST_IMAGES_DIR) if x[-4:]=='.jpg']
TEST_IMAGE_PATHS = [os.path.join(PATH_TO_TEST_IMAGES_DIR, x) for x in images]

In [14]:
# write bboxes info for TableBank(val)
for image_path in tqdm.tqdm(TEST_IMAGE_PATHS):
    item = image_path.split('/')[-1][:-4]
    image = Image.open(image_path).convert('RGB')
    width, height = image.size
    image_np = load_image_into_numpy_array(image)
    image_np_expanded = np.expand_dims(image_np, axis=0)
    output_dict = run_inference_for_single_image(image_np_expanded, detection_graph)
    with open('data_dir/test_images/TableBank/mixed_Detections_v2/'+item+'.txt', "w") as f_out:
        cond = np.where(output_dict['detection_classes']==1)[0]
        boxes = output_dict['detection_boxes'][cond]
        scores = output_dict['detection_scores'][cond]
        for i in range(len(boxes)):
            box = tuple(boxes[i].tolist())
            prob = scores[i]
            ymin, xmin, ymax, xmax = box
            (left, right, top, bottom) = (xmin * width, xmax * width, ymin * height, ymax * height)
            f_out.write(' '.join(['table', str(prob), str(left), str(top), str(right), str(bottom)]))
            f_out.write('\n')

100%|██████████| 1000/1000 [24:40<00:00,  1.48s/it]
